# For-Debugging Notebook

This notebook is a notebook for debugging `unlearning` function, based on the following notebook:
- https://www.kaggle.com/code/eleni30fillou/run-unlearn-finetune

## How to Use

1. implement your `unlearning` function;
2. turn on `internet on` in the right panel;
3. set the variable `USE_MOCK` to `True` in the 2nd code cell;
4. (Optional) modity other parameters in the same cell like `n_checkpoints`;
5. if your codes work,
   - turn off `internet on` in the right panel;
   - set the variable `USE_MOCK` to `False` in the 2nd code cell;
   - save the notebook;
   - and submit!

## Updates
- Ver.5:
  - add a stopwatch decorator
  - make `unlearning` return a updated model
- Ver.4: fix seed

In [1]:
import os
import subprocess
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from torch.utils.data import DataLoader, Dataset

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' 
DEVICE

'cuda'

In [2]:
torch.manual_seed(3047)

Gr = torch.Generator()
Gr.manual_seed(20)

Gf = torch.Generator()
Gf.manual_seed(30)

Gv = torch.Generator()
Gv.manual_seed(40)

In [3]:
# Mock setting

import logging
import requests
import tqdm
from torch.utils.data import Subset
from torchvision import transforms

USE_MOCK: bool = False

if USE_MOCK:
    logging.warning('Running with Mock')
    logging.warning('In this mode, internet access may be required.')

    # The number of checkpoints in this mode.
    # NOTE: 512 checkpoints are required in this competition.
    n_checkpoints = 5
    
    # The directory for a dataset and a pretrained model
    mock_dir = './mock'
    mock_model_path = os.path.join(mock_dir, "weights_resnet18_cifar10.pth")
    os.makedirs(mock_dir, exist_ok=True)

In [4]:
# It's really important to add an accelerator to your notebook, as otherwise the submission will fail.
# We recomment using the P100 GPU rather than T4 as it's faster and will increase the chances of passing the time cut-off threshold.

if DEVICE != 'cuda':
    raise RuntimeError('Make sure you have added an accelerator to your notebook; the submission will fail otherwise!')

In [5]:
# Helper functions for loading the hidden dataset.

if USE_MOCK:
    
    class DatasetWrapper(Dataset):
        
        def __init__(self, ds: Dataset):
            self._ds = ds
    
        def __len__(self):
            return len(self._ds)
    
        def __getitem__(self, index):
            item = self._ds[index]
            result = {
                'image': item[0],
                'image_id': index,
                'age_group': item[1],
                'age': item[1],
                'person_id': index,
            }
            return result
    
    def get_dataset(batch_size, retain_ratio=0.98, thinning_param: int=1, root=mock_dir) -> tuple[DataLoader, DataLoader, DataLoader]:
        
        # utils
        normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        # create dataset
        train_ds = DatasetWrapper(torchvision.datasets.CIFAR10(root=mock_dir, train=True, download=True, transform=normalize))
        retain_ds = Subset(train_ds, range(0, int(len(train_ds)*retain_ratio), thinning_param))
        forget_ds = Subset(train_ds, range(int(len(train_ds)*retain_ratio), len(train_ds), thinning_param))
        val_ds = DatasetWrapper(torchvision.datasets.CIFAR10(root=mock_dir, train=False, download=True, transform=normalize))

        retain_loader = DataLoader(retain_ds, batch_size=batch_size, shuffle=True)
        forget_loader = DataLoader(forget_ds, batch_size=batch_size, shuffle=True)
        validation_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)

        return retain_loader, forget_loader, validation_loader
    
    # For test
#     for sample in get_dataset(3)[0]:
#         print(sample)
#         break
    
else:
    def load_example(df_row):
        image = torchvision.io.read_image(df_row['image_path'])
        result = {
            'image': image,
            'image_id': df_row['image_id'],
            'age_group': df_row['age_group'],
            'age': df_row['age'],
            'person_id': df_row['person_id']
        }
        return result


    class HiddenDataset(Dataset):
        '''The hidden dataset.'''
        def __init__(self, split='train'):
            super().__init__()
            self.examples = []

            df = pd.read_csv(f'/kaggle/input/neurips-2023-machine-unlearning/{split}.csv')
            df['image_path'] = df['image_id'].apply(lambda x: os.path.join('/kaggle/input/neurips-2023-machine-unlearning/', 'images', x.split('-')[0], x.split('-')[1] + '.png'))
            df = df.sort_values(by='image_path')
            df.apply(lambda row: self.examples.append(load_example(row)), axis=1)
            if len(self.examples) == 0:
                raise ValueError('No examples.')

        def __len__(self):
            return len(self.examples)

        def __getitem__(self, idx):
            example = self.examples[idx]
            image = example['image']
            image = image.to(torch.float32)
            example['image'] = image
            return example


    def get_dataset(batch_size):
        '''Get the dataset.'''
        retain_ds = HiddenDataset(split='retain')
        forget_ds = HiddenDataset(split='forget')
        val_ds = HiddenDataset(split='validation')

        retain_loader = DataLoader(retain_ds, batch_size=batch_size, shuffle=True, generator=Gr)
        forget_loader = DataLoader(forget_ds, batch_size=batch_size, shuffle=True, generator=Gf)
        validation_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True, generator=Gv)
        #retain_loader = DataLoader(retain_ds, batch_size=batch_size, shuffle=True)
        #forget_loader = DataLoader(forget_ds, batch_size=batch_size, shuffle=True)
        #validation_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)

        return retain_loader, forget_loader, validation_loader

In [6]:
# Utils
from contextlib import contextmanager
import time

@contextmanager
def stopwatch(name='STOPWATCH'):
    s = time.time()
    try:
        yield
    finally:
        print(f"{name}: {time.time()-s} seconds passed")
        
# for test
# with stopwatch():
#     for i in range(5):
#         time.sleep(1)

In [7]:
from torch.optim.lr_scheduler import CosineAnnealingLR,CosineAnnealingWarmRestarts,StepLR
def kl_loss_sym(x,y):
    kl_loss = nn.KLDivLoss(reduction='batchmean')
    return kl_loss(nn.LogSoftmax(dim=-1)(x),y)
def unlearning(
        net,
        retain_loader,
        forget_loader,
        val_loader,
):
    """Simple unlearning by finetuning."""
    print('-----------------------------------')
    epochs = 8
    retain_bs = 256
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.005,
                          momentum=0.9, weight_decay=0)
    optimizer_retain = optim.SGD(net.parameters(), lr=0.001*retain_bs/64, momentum=0.9, weight_decay=1e-2)
    ##the learning rate is associated with the batchsize we used
    optimizer_forget = optim.SGD(net.parameters(), lr=3e-4, momentum=0.9, weight_decay=0)
    total_step = int(len(forget_loader)*epochs)
    retain_ld = DataLoader(retain_loader.dataset, batch_size=retain_bs, shuffle=True)
    retain_ld4fgt = DataLoader(retain_loader.dataset, batch_size=256, shuffle=True)
    scheduler = CosineAnnealingLR(optimizer_forget, T_max=total_step, eta_min=1e-6)
    if USE_MOCK: ##Use some Local Metric as reference
        net.eval()
        print('Forget')
        evaluation(net, forget_loader, criterion)
        print('Valid')
        evaluation(net, validation_loader, criterion)
    net.train()
    for sample in forget_loader: ##First Stage 
        inputs = sample["image"]
        inputs = inputs.to(DEVICE)
        optimizer.zero_grad()
        outputs = net(inputs)
        uniform_label = torch.ones_like(outputs).to(DEVICE) / outputs.shape[1] ##uniform pseudo label
        loss = kl_loss_sym(outputs, uniform_label) ##optimize the distance between logits and pseudo labels
        loss.backward()
        optimizer.step()
    if USE_MOCK:
        print('Forget')
        evaluation(net,forget_loader,criterion)
        print('Valid')
        evaluation(net, validation_loader,criterion)
        print(f'epoch={epochs} and retain batch_sz={retain_bs}')
    net.train()
    for ep in range(epochs): ##Second Stage 
        net.train()
        for sample_forget, sample_retain in zip(forget_loader, retain_ld4fgt):##Forget Round
            t = 1.15 ##temperature coefficient
            inputs_forget,inputs_retain = sample_forget["image"],sample_retain['image']
            inputs_forget, inputs_retain = inputs_forget.to(DEVICE), inputs_retain.to(DEVICE)
            optimizer_forget.zero_grad()
            outputs_forget,outputs_retain = net(inputs_forget),net(inputs_retain).detach()
            loss = (-1 * nn.LogSoftmax(dim=-1)(outputs_forget @ outputs_retain.T/t)).mean() ##Contrastive Learning loss
            loss.backward()
            optimizer_forget.step()
            scheduler.step()
        for sample in retain_ld: ##Retain Round
            inputs, labels = sample["image"],sample["age_group"]
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer_retain.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_retain.step()
        if USE_MOCK: 
            print(f'epoch {ep}:')
            print('Retain')
            evaluation(net, retain_ld, criterion)
            print('Forget')
            evaluation(net, forget_loader, criterion)
            print('Valid')
            evaluation(net, validation_loader, criterion)
    print('-----------------------------------')
    return net



In [8]:
def evaluation(net, dataloader, criterion, device = 'cuda'): ##evaluation function
    net.eval()
    total_samp = 0
    total_acc = 0
    total_loss = 0.0
    for sample in dataloader:
        images, labels = sample['image'].to(device), sample['age_group'].to(device)
        _pred = net(images)
        total_samp+=len(labels)
        #print(f'total_samp={total_samp}')
        loss = criterion(_pred, labels)
        total_loss += loss.item()
        total_acc+=(_pred.max(1)[1] == labels).float().sum().item()
        #print(f'total_acc={total_acc}')
    #print(f'total_sample={total_samp}')
    mean_loss = total_loss / len(dataloader)
    mean_acc = total_acc/total_samp
    print(f'loss={mean_loss}')
    print(f'acc={mean_acc}')
    return 

In [9]:
import numpy as np
if USE_MOCK:
    
    # NOTE: Almost same as the original codes
    
    # Download
    if not os.path.exists(mock_model_path):
        response = requests.get("https://storage.googleapis.com/unlearning-challenge/weights_resnet18_cifar10.pth")
        open(mock_model_path, "wb").write(response.content)    
    
    os.makedirs('/kaggle/tmp', exist_ok=True)
    retain_loader, forget_loader, validation_loader = get_dataset(64)
    net = resnet18(weights=None, num_classes=10)
    net.to(DEVICE)
    for i in tqdm.trange(n_checkpoints):
        net.load_state_dict(torch.load(mock_model_path))
        net_ = unlearning(net, retain_loader, forget_loader, validation_loader)
        state = net_.state_dict()
        torch.save(state, f'/kaggle/tmp/unlearned_checkpoint_{i}.pth')

    # Ensure that submission.zip will contain exactly 512 checkpoints 
    # (if this is not the case, an exception will be thrown).
    unlearned_ckpts = os.listdir('/kaggle/tmp')
    if len(unlearned_ckpts) != n_checkpoints:
        raise RuntimeError('Expected exactly 512 checkpoints. The submission will throw an exception otherwise.')

    subprocess.run('zip submission.zip /kaggle/tmp/*.pth', shell=True)
    
else:
    if os.path.exists('/kaggle/input/neurips-2023-machine-unlearning/empty.txt'):
        # mock submission
        subprocess.run('touch submission.zip', shell=True)
    else:

        # Note: it's really important to create the unlearned checkpoints outside of the working directory 
        # as otherwise this notebook may fail due to running out of disk space.
        # The below code saves them in /kaggle/tmp to avoid that issue.

        os.makedirs('/kaggle/tmp', exist_ok=True)
        retain_loader, forget_loader, validation_loader = get_dataset(64)
        net = resnet18(weights=None, num_classes=10)
        net.to(DEVICE)
        for i in range(512):
            net.load_state_dict(torch.load('/kaggle/input/neurips-2023-machine-unlearning/original_model.pth'))
            net_ = unlearning(net, retain_loader, forget_loader, validation_loader)
            state = net_.state_dict()
            torch.save(state, f'/kaggle/tmp/unlearned_checkpoint_{i}.pth')

        # Ensure that submission.zip will contain exactly 512 checkpoints 
        # (if this is not the case, an exception will be thrown).
        unlearned_ckpts = os.listdir('/kaggle/tmp')
        if len(unlearned_ckpts) != 512:
            raise RuntimeError('Expected exactly 512 checkpoints. The submission will throw an exception otherwise.')

        subprocess.run('zip submission.zip /kaggle/tmp/*.pth', shell=True)
